confusion matrix :
    http://gcucurull.github.io/caffe/python/deep-learning/2016/06/29/caffe-confusion-matrix/
    https://github.com/gcucurull/caffe-conf-matrix
FCN :
    score :
        https://github.com/shelhamer/fcn.berkeleyvision.org/blob/master/score.py
    infer :
        https://github.com/shelhamer/fcn.berkeleyvision.org/blob/master/infer.py

In [1]:
from __future__ import division
from __future__ import print_function
import caffe
import numpy as np
import os
import sys
from datetime import datetime
from PIL import Image
from IPython.display import Image
from IPython.display import clear_output
import time
import _init_paths

from matplotlib.pyplot import imshow
from matplotlib.pyplot import imread
from matplotlib.pyplot import imsave
%matplotlib inline

In [2]:
def log_progress(sequence, title='', every=1, size=None, end_msg='', removeAtTheEnd=False):
    from ipywidgets import IntProgress, HTML, VBox
    from IPython.display import display

    is_iterator = False
    if size is None:
        try:
            size = len(sequence)
        except TypeError:
            is_iterator = True
    if size is not None:
        if every is None:
            if size <= 200:
                every = 1
            else:
                every = int(size / 200)     # every 0.5%
    else:
        assert every is not None, 'sequence is iterator, set every'

    if is_iterator:
        progress = IntProgress(min=0, max=1, value=1)
        progress.bar_style = 'info'
    else:
        progress = IntProgress(min=0, max=size, value=0)
    label = HTML()
    box = VBox(children=[label, progress])
    display(box)

    index = 0
    try:
        for index, record in enumerate(sequence, 1):
            if index == 1 or index % every == 0:
                if is_iterator:
                    label.value = '{index} / ?'.format(index=index)
                else:
                    progress.value = index
                    if isinstance(sequence, dict):
                        val = ''
                    else:
                        val=sequence[index-1]
                            
                    label.value = u'{title} : {value} ({index} / {size})'.format(
                        title=title,
                        value=val,
                        index=index,
                        size=size
                    )
            yield record
    except:
        progress.bar_style = 'danger'
        raise
    else:
        progress.bar_style = 'success'
        progress.value = index
        label.value = end_msg
        if removeAtTheEnd:
            box.close()
    

In [3]:
SHRECmodels = '/home/jguerry/workspace/SHRECmodels/'
# prototxt_path = SHRECmodels+'c14_kf3_var5_train_val.prototxt'
# weights_path =  SHRECmodels+'c14_kf3_var5.caffemodel'

prototxt_path = SHRECmodels+'14bi28_kf5_var5 .prototxt'
weights_path =  SHRECmodels+'14bi28_kf5_var5.caffemodel'



caffe_gpu = True
if not caffe_gpu:
    caffe.set_mode_cpu()
else:
    caffe.set_mode_gpu()
    caffe.set_device(0);

In [4]:
# net = caffe.Net(prototxt_path, caffe.TEST)
net = caffe.Net(prototxt_path, weights_path, caffe.TEST)

In [13]:
results={}
results['id_gesture']=[]
results['id_finger']=[]
results['id_subject']=[]
results['id_essai']=[]
results['label_gt']=[]
results['label_found']=[]
results['label_gt_28']=[]
results['label_found_28']=[]
results['time_millisecond']=[]
                     

In [14]:
print("###############################################################")

# net.layers[0].images_processed = 100

for idx in log_progress(range(net.layers[0].nb_images), title='iteration', end_msg='Finished !',removeAtTheEnd=True):
# for idx in log_progress(range(5), title='iteration', end_msg='Finished !',removeAtTheEnd=True):

    #print("Idx :",idx)
    t = time.time()
    net.forward()
    results['id_gesture'].append(int(net.layers[0].list_images[idx,0]))
    results['id_finger'].append(int(net.layers[0].list_images[idx,1]))
    results['id_subject'].append(int(net.layers[0].list_images[idx,2]))
    results['id_essai'].append(int(net.layers[0].list_images[idx,3]))
    results['label_gt'].append(int(net.blobs['label'].data[0]))
    results['label_found'].append(int(net.blobs['fc8'].data[0].argmax(0)))
    results['label_gt_28'].append(int(net.blobs['label_28'].data[0]))
    results['label_found_28'].append(int(net.blobs['fc_28'].data[0].argmax(0)))
    results['time_millisecond'].append(round(100000*(time.time() - t))/100)




###############################################################


In [7]:
def fast_hist(a, b, n):
    k = (a >= 0) & (a < n)
    return np.bincount(n * a[k].astype(int) + b[k], minlength=n**2).reshape(n, n)

In [17]:
hist = fast_hist(np.asarray(results['label_gt']),np.asarray(results['label_found']),14)


acc = np.diag(hist).sum() / hist.sum()
# per-class accuracy
pc_acc = np.diag(hist) / hist.sum(1)
mean_accuracy= np.nanmean(pc_acc)

print("Accuracy:",acc)
print("pc_acc:",pc_acc)
print("mean_accuracy:",mean_accuracy)

Accuracy: 0.810714285714
pc_acc: [ 0.84482759  0.49180328  0.65454545  0.80392157  0.72727273  0.9137931
  0.96774194  0.81481481  0.79411765  0.90163934  0.75362319  0.9122807
  0.96551724  0.80821918]
mean_accuracy: 0.811008412145


In [18]:
hist = fast_hist(np.asarray(results['label_gt_28']),np.asarray(results['label_found_28']),28)


acc = np.diag(hist).sum() / hist.sum()
# per-class accuracy
pc_acc = np.diag(hist) / hist.sum(1)
mean_accuracy= np.nanmean(pc_acc)

print("Accuracy:",acc)
print("pc_acc:",pc_acc)
print("mean_accuracy:",mean_accuracy)

Accuracy: 0.702380952381
pc_acc: [ 0.75        0.8         0.42424242  0.46428571  0.48148148  0.71428571
  0.62962963  0.79166667  0.34482759  0.69230769  0.53125     0.61538462
  0.60606061  0.93103448  0.65384615  0.75        0.75        0.83333333
  0.67741935  0.83333333  0.45454545  0.83333333  0.71428571  0.96551724
  0.89285714  0.96666667  0.76315789  0.77142857]
mean_accuracy: 0.701292171711
